In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch
!pip install kogpt2_transformers
!pip install kss
!pip install mecab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.4-py3-none-any.whl size=54464 sha256=5a2de4a034a611cfb6d31b4f45ae06cfff7eb46cff84c4f44385cb9056a0091a
  Stored in directory: /root/.cache/pip/wheels/61/7b/ba/e620ef5d96a61cdd83bdee4c2bb4aec8a74de5d72fcbb00e80
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646665 sha256=1594ade83cdabcb992a48250b6967035a03d8c61f8a157fefbf95f762cd33c45
  Stored in directory: /root/.cache/pip/wheels/5c/6f/b4/ab61b8863d7d8b1409def8ae31adcaa089fa91b8d022ec309d
Successfully built kss pecab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel 

In [ ]:
%run /content/drive/MyDrive/my_ws/project/aischool-final/dialogLM/model/kogpt2.py

In [ ]:
%cd /content/drive/MyDrive//my_ws/project/aischool-final/dialogLM

/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM


In [ ]:
######################## evaluate를 위해 test문장 생성  ##################################
import os
import numpy as np
import torch

import sys
sys.path.append('/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM')

from model.kogpt2 import DialogKoGPT2
from kogpt2_transformers import get_kogpt2_tokenizer

import kss

root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
data_path = f"{root_path}/data/wellness_dialog_for_autoregressive_test.txt"
save_ckpt_path = f"{root_path}/checkpoint/kogpt2-wellnesee-auto-regressive_epoch7_origin.pth"
gener_path = f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.txt"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

with open(data_path, 'r', encoding='utf-8') as read_file:
  with open(gener_path, 'w', encoding='utf-8') as gener_file:
    while True:
      line = read_file.readline()
      if not line:
        break
      datas = line.split("    ")
      # print(datas[0])  # 유저발화
      # print(datas[1][:-1])  # 챗봇발화
      sent = datas[0]
      tokenized_indexs = tokenizer.encode(sent)  # 입력받은 문장을 토크나이저로 인코드함.
      input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)  #
      sample_output = model.generate(input_ids=input_ids)
      '''
      tensor([[    0,  1447, 22296,  3973, 47855, 47059, 47855, 42280, 45664, 17513,
              16991, 19991, 47440,     1,     0, 30790,  5680, 47438,   199,   737,
                215,  2747,  4171,  3161,   779,   241, 13996, 47440,     1,     0]])
      '''
      out_sent = tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True)
      # kss 사용해서 text split
      if out_sent:
        sentences = kss.split_sentences(out_sent)
        selected_fir_sentence = sentences[0]
        if len(sentences) == 1:
          selected_sec_sentence = ''
        else:
          selected_sec_sentence = sentences[1] if sentences[0] != sentences[1] else ''
        answer = selected_fir_sentence + selected_sec_sentence
      else:
        answer = ''
      print(answer)
      gener_file.write(f'{answer}\n')
  gener_file.close()
read_file.close()


In [ ]:
############ 빠른 작업을 위해 excel형태로 변환 #####################

import pandas as pd

root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
data_file = f"{root_path}/data/wellness_dialog_for_autoregressive_eval_test.txt"

def text_to_excel(input_file, output_file):
    # 텍스트 파일 읽기
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # 각 줄을 엑셀 한 행으로 변환
    data = {'answer': lines}

    # pandas DataFrame 생성
    df = pd.DataFrame(data)

    # 엑셀 파일로 저장
    df.to_excel(output_file, index=False)

if __name__ == "__main__":
    # 텍스트 파일 경로 및 엑셀 파일 경로 설정
    input_text_file = f"{root_path}/data/wellness_dialog_for_autoregressive_eval_test.txt"
    output_excel_file = f"{root_path}/data/wellness_dialog_for_autoregressive_eval_test.xlsx"

    # 함수 호출
    text_to_excel(input_text_file, output_excel_file)

    print(f'Text file "{input_text_file}" has been converted to Excel file "{output_excel_file}".')

Text file "/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM/data/wellness_dialog_for_autoregressive_eval_test.txt" has been converted to Excel file "/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM/data/wellness_dialog_for_autoregressive_eval_test_7.xlsx".


In [ ]:
############ 빠른 작업을 위해 excel형태로 변환 #####################

import pandas as pd

root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
gener_path = f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.txt"
output_file = f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.xlsx"

def text_to_excel(input_file, output_file):
    # 텍스트 파일 읽기
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # 각 줄을 엑셀 한 행으로 변환
    data = {'answer': lines}

    # pandas DataFrame 생성
    df = pd.DataFrame(data)

    # 엑셀 파일로 저장
    df.to_excel(output_file, index=False)

if __name__ == "__main__":
    # 텍스트 파일 경로 및 엑셀 파일 경로 설정
    input_text_file = f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.txt"
    output_excel_file = f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.xlsx"

    # 함수 호출
    text_to_excel(input_text_file, output_excel_file)

    print(f'Text file "{input_text_file}" has been converted to Excel file "{output_excel_file}".')

Text file "/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM/data/wellness_dialog_for_autoregressive_gener7.txt" has been converted to Excel file "/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM/data/wellness_dialog_for_autoregressive_gener7.xlsx".


In [ ]:
# root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
# data_path = f"{root_path}/data/wellness_dialog_for_autoregressive_test.txt"
# eval_path = f"{root_path}/data/wellness_dialog_for_autoregressive_eval_test.txt"

# with open(data_path, 'r', encoding='utf-8') as read_file:
#   with open(eval_path, 'w', encoding='utf-8') as eval_file:
#     while True:
#       line = read_file.readline()
#       if not line:
#         break
#       datas = line.split("    ")
#       eval_file.write(f'{datas[1][:-1]}\n')
#   eval_file.close()
# read_file.close()

In [ ]:
# # 해당 질문이 있는 카테고리안에 대답을 모두 reference로 가져오기 --- 진행 못함.
# import pandas as pd
# pd.ExcelWriter()

# root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
# data_path = f"{root_path}/data/wellness_dialog_for_autoregressive_test.txt"
# eval_path = f"{root_path}/data/wellness_dialog_for_autoregressive_eval_cate_answer.xlsx"


# with open(data_path, 'r', encoding='utf-8') as read_file:
#   with open(eval_path, 'w', encoding='utf-8') as eval_file:
#     while True:
#       line = read_file.readline()
#       if not line:
#         break
#       question = line.split("    ")[0]

#       eval_file.write(f'{datas[1][:-1]}\n')
#   eval_file.close()
# read_file.close()


In [ ]:
################## 평가하기  -  BLEU SCORE  #####################

import pandas as pd
import nltk
from nltk.translate.bleu_score import SmoothingFunction, modified_precision, sentence_bleu

gener = pd.read_excel(f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.xlsx")
answer = pd.read_excel(f"{root_path}/data/wellness_dialog_for_autoregressive_eval_test.xlsx")

bleu_score_list=[]
for row in range(len(gener)):
  gene_sentence = gener['answer'][row]
  candidate = kss.split_sentences(gene_sentence)[0] if gene_sentence else ''
  ans_sentence = answer['answer'][row]
  references = kss.split_sentences(ans_sentence)[0]
  # print(candidate,references)
  bleu_score = nltk.translate.bleu_score.sentence_bleu(references, candidate)
  bleu_score_list.append(bleu_score)
  print(bleu_score)
avg_bleu = np.mean(bleu_score_list)
print("BLEU Score:", avg_bleu)



/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

1.091900080306308e-231
1.2083860769943533e-231
1.3483065280626046e-231
1.6034157163765524e-231
1.6034157163765524e-231
1.419366116189521e-231
1.2340561512781763e-231
1.384292958842266e-231
1.419366116189521e-231
1.2340561512781763e-231
1.2340561512781763e-231
1.331960397810445e-231
1.331960397810445e-231
1.384292958842266e-231
1.331960397810445e-231
1.384292958842266e-231
1.419366116189521e-231
1.384292958842266e-231
1.331960397810445e-231
1.419366116189521e-231
1.384292958842266e-231
1.7280875057645142e-231
1.2340561512781763e-231
1.6034157163765524e-231
1.384292958842266e-231
1.419366116189521e-231
1.331960397810445e-231
1.419366116189521e-231
1.6034157163765524e-231
1.6034157163765524e-231
1.384292958842266e-231
1.384292958842266e-231
1.331960397810445e-231
1.331960397810445e-231
1.331960397810445e-231
1.384292958842266e-231
1.3416480207402436e-231
1.3048555118335446e-231
1.5057747795382668e-231
1.2627076138080564e-231
1.3568676344828118e-231
1.4147351699132998e-231
1.28822975391941

In [ ]:
################## 평가하기  -  METEOR #####################

import pandas as pd
from nltk.translate.meteor_score import meteor_score
nltk.download('wordnet')

gener = pd.read_excel(f"{root_path}/data/wellness_dialog_for_autoregressive_gener7.xlsx")
answer = pd.read_excel(f"{root_path}/data/wellness_dialog_for_autoregressive_eval_test.xlsx")

meteor_score_list=[]
for row in range(len(gener)):
  gene_sentence = gener['answer'][row]
  candidate = kss.split_sentences(gene_sentence)[0] if gene_sentence else ''
  ans_sentence = answer['answer'][row]
  # references = [kss.split_sentences(ans_sentence)[0], kss.split_sentences(ans_sentence)[1] if len(kss.split_sentences(ans_sentence)) >1 else kss.split_sentences(ans_sentence)[0]]
  references = [kss.split_sentences(ans_sentence)[0]]
  meteor_score_value = meteor_score([references], [candidate])
  # meteor_score_value = meteor_score([[ans_sentence]], [gene_sentence])  # count = 1, meteor = 0.5
  if meteor_score_value>0.1:
    meteor_score_list.append(meteor_score_value)
  else:
    continue
  print(meteor_score_value)
avg_meteor = np.mean(meteor_score_list)
print(f'meteor Score: {avg_meteor}, count : {len(meteor_score_list)}')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
meteor Score: 0.5, count : 61


In [ ]:
########################### 답변 간이로 확인 ###########################


import os
import numpy as np
import torch

import sys
sys.path.append('/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM')

from model.kogpt2 import DialogKoGPT2
from kogpt2_transformers import get_kogpt2_tokenizer

import kss

root_path = '/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM'
# data_path = f"{root_path}/data/wellness_dialog_for_autoregressive_train.txt"
save_ckpt_path = f"{root_path}/checkpoint/kogpt2-wellnesee-auto-regressive_epoch7_origin.pth"
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

sent = 1
while sent:
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  if sent != '종료':  # 종료가 아니면 진행
    tokenized_indexs = tokenizer.encode(sent)  # 입력받은 문장을 토크나이저로 인코드함.
    input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)  #
    sample_output = model.generate(input_ids=input_ids)
    out_sent = tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True)

    # kss 사용해서 text split

    sentences = kss.split_sentences(out_sent)
    selected_fir_sentence = sentences[0]
    selected_sec_sentence = sentences[1] if sentences[0] != sentences[1] else ''

    print("Answer: " + selected_fir_sentence + selected_sec_sentence)
    print(100 * '-')

  else:
        break
